# 特征

2021年9月16日，漱玉平民出现日线仙人指路形态。换手率18.4%，上涨4.4%，上影线5.6%。日线量比达4.8，30分钟量比更是高达29.5倍。其后不久，连续出现20cm涨停。


<img src="https://images.jieyu.ai/images/202110/20211103195208.png" width="400"/>

买入点特征：
1. 近期(5日内）出现日线量比3倍以上，30分钟量比10倍以上。
2. 当前30分钟出现底部特征：长下影、RSI超卖
3. 低位: 120日wr < 0.3

In [3]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
import datetime
await init_notebook()

In [ ]:
def fire_long(code, bars, **params):
    altitude_threshold = params.get("altitude", 0.6)
    turnover_threshold = params.get("turnover", 0.07)
    lb_threshold = params.get("lb", 3)
    pcr_threshold = params.get("pcr", 0.05)
    
    close = bars["close"]
    volume = bars["volume"]
    
    minv = np.min(volume[-6:-1])
    high_price = np.max(close)
    pcr = close[-1] / close[-2] - 1
    is_down = bars[-1]["close"] <= bars[-1]["open"]
    
    if is_down:
        return None

    altitude = close[-1] / high_price 
    lb = volume[-1] / minv
    to = jq_get_turnover_realtime(code, volume[-1], close[-1])
    
    if altitude < altitude_threshold and to > turnover_threshold and \
    lb > lb_threshold and pcr <= pcr_threshold:
        return bars[-1]["frame"], bars[-1]["close"], altitude, lb, to, pcr
    else:
        return None

In [ ]:
def jq_scan_v0(lb=3, turnover=0.07, altitude=0.6, pcr=0.05):
    """直接使用jq数据进行扫描，仅适用快速搜索当日行情"""
    params = {
        "lb": lb,
        "turnover": turnover,
        "altitude": altitude,
        "pcr": pcr
    }
    
    results = []
    for code in jq_choose_stocks():
        name = jq_get_name(code)
        
        bars = jq_get_bars(code, 120)
        signal = fire_long(code, bars, **params)
        if signal is None:
            continue
        
        frame, price, altitude, lb, to, pcr_0 = signal
        results.append([name, code, lb, to, altitude, pcr_0])

    df = pd.DataFrame(results, columns=["name", "code", "lb", "turnover", "altitude", "pcr"])

    mail_notify(f"[{arrow.now().date()}]低位放量", "低位放量 ", params, df)
    return df

df = scan_v0()
df

# V1 低位放量

In [ ]:
async def backtest_v1(start, end):
    start = arrow.get(start).date()
    end = arrow.get(end).date()
    
    signals = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)

            for i in range(120, len(bars) - 15):
                xbars = bars[i-120:i]

                signal = fire_long(code, xbars, pcr=0.2)
                if signal is None:
                    continue
                else:
                    frame, price, altitude, lb, to, pcr_0 = signal
                    row = [sec.display_name, code, frame, price, altitude, lb, to, pcr_0]
                    for j in [1, 3, 5, 10, 15]:
                        ybars = bars[i:i+j]
                        yclose = np.max(ybars["close"])
                        row.append(yclose/price-1)

                    signals.append(row)
        except Exception:
            pass
        
    columns = ["name", "code", "frame", "price", "altitude", "lb", "to", "pcr_0", 
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(signals, columns=columns)
        
df = await backtest_v1("2021-01-04", "2021-09-30")
df

In [ ]:
df.describe()

# V2 加上次新股属性

回测表明，次新股低位放量后，10交易日平均涨幅为12%；15日平均涨幅为15%。

In [ ]:
async def backtest_v2(start, end, **params):
    end = arrow.get(end).date()
    start = arrow.get(start).date()
    
    signals = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        
        if tf.day_shift(sec.ipo_date, 60) < start:
            continue
            
        try:
            bars = await sec.load_bars(sec.ipo_date, end, FrameType.DAY)

            for i in range(60, min(250, len(bars) - 15)): #限制在一年内
                xbars = bars[i-60:i]

                signal = fire_long(code, xbars, **params)
                if signal is None:
                    continue
                else:
                    frame, price, altitude, lb, to, pcr_0 = signal
                    row = [sec.display_name, code, frame, price, altitude, lb, to, pcr_0]
                    for j in [1, 3, 5, 10, 15]:
                        ybars = bars[i:i+j]
                        yclose = np.max(ybars["close"])
                        row.append(yclose/price-1)

                    signals.append(row)
        except Exception:
            pass
        
    columns = ["name", "code", "frame", "price", "altitude", "lb", "to", "pcr_0", 
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(signals, columns=columns)

In [ ]:
df = await backtest_v2("2015-01-04", "2021-09-30", altitude=0.6, turnover=0.07, lb=3, pcr=0.05)
df.describe()

In [ ]:
report("低位放量v2次新股版回测结果", {"altitude":0.6, "turnover":0.07, "lb":3, "pcr":0.05}, result_df=df)

# v3
放量后，缩量下跌至低于放量涨当日开盘价买入，计算持有1，3，5，10，15日，以收盘价卖出的利润。结果表明并无改进。

In [ ]:
async def backtest_v3(start, end):
    start = arrow.get(start).date()
    end = arrow.get(end).date()
    
    signals = []
    
    # 寻找发出放量信号的个股
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)

            for i in range(120, len(bars) - 30):
                xbars = bars[i-120:i]

                signal = fire_long(code, xbars, pcr=0.3, altitude=0.8, lb=2.5)
                if signal is None:
                    continue
                else:
                    frame, close, altitude, lb, to, pcr_0 = signal
                    row = [code, frame, altitude, lb, to]
                    signals.append(row)
        except Exception:
            pass
        
    # 如果股价回到放量日开盘价以下，则为买入点
    results = []
    for code, sig_start, altitude, lb, to in signals:
        sig_end = tf.day_shift(sig_start, 15)
        sec = Security(code)
        try:
            bars = await sec.load_bars(sig_start, sig_end, FrameType.DAY)
            open_price = bars["open"][0]
            
            sig_bars = bars[1:]
            pos = np.argwhere(sig_bars["low"] <= open_price).flatten()
            if len(pos) == 0:
                continue
                
            order_day = sig_bars[pos[0]]["frame"]
            ystart = tf.day_shift(order_day, 1)
            yend = tf.day_shift(order_day, 16)
            
            ybars = await sec.load_bars(ystart, yend, FrameType.DAY)
            
            row = [sec.display_name, code, sig_start, order_day, open_price, altitude, lb, to]
            for n in [1, 3, 5, 10, 15]:
                c = ybars[n]["close"] / open_price - 1
                row.append(c)
            
            results.append(row)
        except Exception as e:
            break
        
    columns = ["name", "code", "fire_date", "oder_date", "order_price", "altitude", "lb", "to",  
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(results, columns=columns)
        
df = await backtest_v3("2021-01-04", "2021-09-30")
df

# V4
找出近10天日线放量最大超过4倍，或者换手率超过20%的标的，30分钟线出现rsi低于25%时发出提示。

In [ ]:
def daybars_trigger(name, code, bars, results, frame_type):
    n = 10
    last_n = bars[-n:]
    volume = last_n["volume"]
    vr = max(volume)/min(volum)
#     if vr < 4:
#         return
    
    pos = np.argmax(volume)
    flag = last_n[pos]["close"] < last_n[pos]["open"] #阴线放量
    
    results.append([name, code, vr, n-pos, flag])
    
result = await scan(daybars_trigger, 15)

In [ ]:
result